In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import numpy as np
import csv
import string
import requests
import io
from zipfile import ZipFile
from tensorflow.contrib import learn
sess = tf.Session()



In [2]:
zip_url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
r=requests.get(zip_url)
z = ZipFile(io.BytesIO(r.content))
file = z.read('SMSSpamCollection')
text_data = file.decode()
text_data = text_data.encode('ascii',errors='ignore')
text_data = text_data.decode().split('\n')
text_data = [x.split('\t') for x in text_data if len(x)>=1]

In [4]:
texts = [x[1] for x in text_data]
target = [x[0] for x in text_data]
target = [1 if x=='spam' else 0 for x in target]

In [5]:
texts=[ x.lower() for x in texts]  #所有word变成小写
texts=[''.join(c for c in x if c not  in string.punctuation) for x in texts] #除去所有标点
texts=[''.join(c for c in x if c not  in "0123456789") for x in texts] 
texts = [' '.join(x.split()) for x in texts]


In [6]:
sentence_size = 25 #每一个句子所包含的最大单词数
min_word_freq = 3
vocab_processor=learn.preprocessing.VocabularyProcessor(sentence_size,min_frequency=min_word_freq)
vocab_processor.fit_transform(texts)
embedding_size=len(vocab_processor.vocabulary_) #2108 表示有2108 个单词

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.


In [7]:
train_indices = np.random.choice(len(texts), round(len(texts)*0.8), replace=False)
test_indices = np.array(list(set(range(len(texts))) - set(train_indices)))
texts_train = [x for ix, x in enumerate(texts) if ix in train_indices]
texts_test = [x for ix, x in enumerate(texts) if ix in test_indices]
target_train = [x for ix, x in enumerate(target) if ix in train_indices]
target_test = [x for ix, x in enumerate(target) if ix in test_indices]

In [10]:
identity_mat = tf.diag(tf.ones(shape=[embedding_size]))
#implement a logistic regression
A = tf.Variable(tf.random_normal(shape=[embedding_size,1]))  #单词的稀疏向量相加得到最后的向量为2108*1 维
b = tf.Variable(tf.random_normal(shape=[1,1]))
x_data = tf.placeholder(shape=[sentence_size], dtype=tf.int32)
y_target = tf.placeholder(shape=[1, 1], dtype=tf.float32)

Instructions for updating:
Colocations handled automatically by placer.


In [21]:
x_embed = tf.nn.embedding_lookup(identity_mat, x_data)
x_col_sums = tf.reduce_sum(x_embed, 0)
# 模型计算
x_col_sums_2D = tf.expand_dims(x_col_sums, 0)
model_output = tf.add(tf.matmul(x_col_sums_2D, A), b)
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=model_output, labels=y_target))
perdiction=tf.sigmoid(model_output)
my_opt=tf.train.GradientDescentOptimizer(0.001)
train_step=my_opt.minimize(loss)

In [22]:
init = tf.initialize_all_variables()
sess.run(init)

In [23]:
loss_vec = []
train_acc_all = []
train_acc_avg = []
# ix 为texts_train的中的每一个sentence的index， t 为texts_train的中的每一个sentence的每一个word对应vocabulary的index
for ix,t in enumerate(vocab_processor.fit_transform(texts_train)):
    y_data = [[target_train[ix]]]
    sess.run(train_step, feed_dict={x_data: t, y_target: y_data})
    temp_loss = sess.run(loss, feed_dict={x_data: t, y_target: y_data})
    loss_vec.append(temp_loss)
    if (ix+1)%100==0:
        print('Training Observation #' + str(ix+1) + ': Loss = ' + 
        str(temp_loss))
    [[temp_pred]] = sess.run(perdiction, feed_dict={x_data:t, y_target:y_data})
    train_acc_temp = target_train[ix]==np.round(temp_pred)
    train_acc_all.append(train_acc_temp)
    if len(train_acc_all) >= 50:
     train_acc_avg.append(np.mean(train_acc_all[-50:]))


Training Observation #10: Loss = 9.847071e-07
Training Observation #20: Loss = 15.392619
Training Observation #30: Loss = 5.7623324
Training Observation #40: Loss = 10.175142
Training Observation #50: Loss = 9.701462
Training Observation #60: Loss = 6.3985972
Training Observation #70: Loss = 1.2686572
Training Observation #80: Loss = 6.8527193
Training Observation #90: Loss = 0.012949951
Training Observation #100: Loss = 0.83515745
Training Observation #110: Loss = 5.6375513
Training Observation #120: Loss = 1.1890194
Training Observation #130: Loss = 0.1439481
Training Observation #140: Loss = 0.008554426
Training Observation #150: Loss = 3.2505789
Training Observation #160: Loss = 0.0018735728
Training Observation #170: Loss = 0.030016717
Training Observation #180: Loss = 10.104113
Training Observation #190: Loss = 0.0053375377
Training Observation #200: Loss = 3.8642669
Training Observation #210: Loss = 0.004516901
Training Observation #220: Loss = 0.0019680755
Training Observation 

In [29]:
print(train_acc_avg[-1]) # 84% train
test_acc_all=[]
for ix,t in enumerate(vocab_processor.fit_transform(texts_test)):
    y_data=[[target_test[ix]]]
    [[temp_pred]] = sess.run(perdiction, feed_dict={x_data:t, y_target:y_data})
    test_acc_temp = target_test[ix]==np.round(temp_pred)
    test_acc_all.append(test_acc_temp)
print(np.mean(test_acc_all)) #78%

0.84
0.768609865470852
